<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/NorthwesternHeader.png?raw=1">

### Analyze AG_NEWS_SUBSET Data <br>

AG is a collection of more than 1 million news articles. News articles have been gathered from more than 2000 news sources by ComeToMyHead in more than 1 year of activity. ComeToMyHead is an academic news search engine which has been running since July, 2004. The dataset is provided by the academic comunity for research purposes in data mining (clustering, classification, etc), information retrieval (ranking, search, etc), xml, data compression, data streaming, and any other non-commercial activity.<br> 

For more information, please refer to the link http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html<br> 


The AG's news topic classification dataset is constructed by choosing 4 largest classes (**World**, **Sports**, **Business**, and **Sci/Tech**) from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600.<br>

Homepage: https://arxiv.org/abs/1509.01626<br>

Source code: tfds.text.AGNewsSubset

Versions:

1.0.0 (default): No release notes.
Download size: 11.24 MiB

Dataset size: 35.79 MiB

## References
1. Deep Learning with Python, Francois Chollet (https://learning.oreilly.com/library/view/deep-learning-with/9781617296864/)
 * Chapter 10: Deep learning for time series
 * Chapter 11: Deep learning for text
2. Deep Learning A Visual Approach, Andrew Glassner (https://learning.oreilly.com/library/view/deep-learning/9781098129019/)
 * Chapter 19: Recurrent Neural Networks
 * Chapter 20: Attention and Transformers

## The Transformer Architecture

## Understanding Self-Attention

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/SelfAttention.png?raw=1">

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/DogAte.png?raw=1">

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/InputOutput.png?raw=1">

## Generalized Self-Attention: The Query-Key-Value Model

#### Retrieving images from a database: the “query” is compared to a set of “keys,” and the match scores are used to rank “values” (images).

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/DogsBeach.png?raw=1">

## Multi-Head Attention

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/MultiHead.png?raw=1">

## Import Packages

In [1]:
import re
import string
from packaging import version
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow import keras

## Verify TensorFlow Version and Keras Version

In [2]:
print("This notebook requires TensorFlow 2.0 or above")
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >=2

This notebook requires TensorFlow 2.0 or above
TensorFlow version:  2.8.0


In [3]:
print("Keras version: ", keras.__version__)

Keras version:  2.8.0


## Mount Google Drive to Colab Environment

In [4]:
# from google.colab import drive
# drive.mount('/content/gdrive')

## The Transformer Encoder

## Load Data

In [5]:
# register  ag_news_subset so that tfds.load doesn't generate a checksum (mismatch) error
!python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums --datasets=ag_news_subset

dataset, info = tfds.load('ag_news_subset', with_info=True,  split=['train[:95%]','train[95%:]', 'test'],batch_size = 32, as_supervised=True)
train_ds, val_ds, test_ds = dataset

I0426 18:11:32.754151 139855582869376 download_and_prepare.py:200] Running download_and_prepare for dataset(s):
ag_news_subset
2022-04-26 18:11:32.960419: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: HTTP response code 404".
I0426 18:11:33.354961 139855582869376 dataset_info.py:434] Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: ag_news_subset/1.0.0
I0426 18:11:34.044365 139855582869376 dataset_info.py:361] Load dataset info from /tmp/tmp2l49c9tstfds
I0426 18:11:34.046195 139855582869376 download_and_prepare.py:138] download_and_prepare for dataset ag_news_subset/1.0.0...
I0426 18:11:34.046568 139855582869376 dataset_builder.py:357] Generating dataset ag_news_subset (/root/tensorflow

## Preparing Text Data

In [6]:
text_only_train_ds = train_ds.map(lambda x, y: x)

## Vectorizing Data

In [7]:
max_length = 52
max_tokens = 1000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

## Transformer Encoder Implemented As Subclassed `Layer`

In [8]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/TransformerEncoder.png?raw=1">

## Using Transformer Encoder For Text Classification

In [9]:
vocab_size = 1000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(4, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="SparseCategoricalCrossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         256000    
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 1028  

## Training and Evaluating Transformer Encoder Based Model

In [11]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
    ]
                                      
model.fit(int_train_ds, validation_data=int_val_ds, epochs=200, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/200
3563/3563 [==============================] - 31s 7ms/step - loss: 0.5174 - accuracy: 0.8153 - val_loss: 0.3996 - val_accuracy: 0.8552
Epoch 2/200
3563/3563 [==============================] - 24s 7ms/step - loss: 0.4180 - accuracy: 0.8518 - val_loss: 0.3897 - val_accuracy: 0.8598
Epoch 3/200
3563/3563 [==============================] - 25s 7ms/step - loss: 0.4009 - accuracy: 0.8569 - val_loss: 0.3837 - val_accuracy: 0.8637
Epoch 4/200
3563/3563 [==============================] - 24s 7ms/step - loss: 0.3910 - accuracy: 0.8598 - val_loss: 0.3820 - val_accuracy: 0.8648
Epoch 5/200
3563/3563 [==============================] - 24s 7ms/step - loss: 0.3832 - accuracy: 0.8637 - val_loss: 0.3868 - val_accuracy: 0.8613
Epoch 6/200
3563/3563 [==============================] - 24s 7ms/step - loss: 0.3769 - accuracy: 0.8661 - val_loss: 0.3967 - val_accuracy: 0.8618
Epoch 7/200
238/238 [==============================] - 1s 5ms/step - loss: 0.4045 - accuracy: 0.8538
Test acc: 0.854


## Using Positional Encoding to Re-Inject Order Information

## Implementing Positional Embedding As Subclassed Layer

In [12]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

## Putting it all together: A text-classification Transformer

## Combining Transformer Encoder with Positional Embedding

In [13]:
vocab_size = 1000
sequence_length = 52
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",save_best_only=True)
    ,tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=200, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        269312    
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder_1 (Tran  (None, None, 256)        543776    
 sformerEncoder)                                                 
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                           

## When To Use Sequence Models over Bag-of-Words Models

<img src="https://github.com/djp840/MSDS_458_Public/blob/master/images/SequenceVBagWords.png?raw=1">